# BERT for yelp prediction
Recall our goals: 1). Find what attributes are ``important'' in sentiment analysis; 2). Predict ratings based on features.
Adapted from
https://towardsdatascience.com/bert-for-dummies-step-by-step-tutorial-fb90890ffe03

# Preprocessing
We preprocess data into numpy format

In [5]:
import numpy as np
import pandas as pd

class data:
    def __init__(self, fileName):
        df = pd.read_csv(fileName)
        values = df.values
        # Star ratings, notice for test/validation data, these are actually ids
        self.stars = np.array(values[:, 0], dtype = int)
        self.stars -= 1
        # Name of restaurants
        self.names = values[:, 1]
        # Text of review
        self.text = values[:, 2]
        # Date of review
        self.date = values[:, 3]
        # City of restaurant
        self.city = values[:, 7]
        # Misc categories, might not use it
        self.category = values[:, 10]
        # Sentiment score
        self.sentiment = np.nan_to_num(values[:, 13])
        s = {0, 1, 2, 3, 7, 10}
        cols = []
        for i in range(values.shape[1]):
            if i not in s: cols.append(i)
        # Other numerical measurements
        self.numerical = np.array(values[:, cols], dtype = np.float64)
        self.text_df = None
    
    def center(self):
        # Center all numerical attributes to mean 0
        mean = np.nanmean(self.numerical, axis = 0)
        self.numerical -= mean
        self.numerical = np.nan_to_num(self.numerical)

    '''def transfer_text(self):
        self.text_df = pd.DataFrame({
            "id": range(len(self.text)),
            "label": self.stars,
            "alpha": ["a"] * len(self.text),
            "text": self.text
        })
        self.text_df["text"] = self.text_df["text"].replace(r"\n", " ", regex = True)'''

    

train_data = data("Yelp_train.csv")
validation_data = data("Yelp_validate.csv")
test_data = data("Yelp_test.csv")



# Setup BERT
Here we import all modules BERT needs, preprocess text data into desired
format

In [6]:
!pip3 install pytorch_pretrained_bert

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt


'''device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)'''

Using TensorFlow backend.


'device = torch.device("cuda" if torch.cuda.is_available() else "cpu")\nn_gpu = torch.cuda.device_count()\ntorch.cuda.get_device_name(0)'

In [7]:
# Add proper prefix/suffix to each sentence
sentences = ["[CLS] " + text + " [SEP]" for text in train_data.text]
# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-cased", do_lower_case = False)
# Tokenize each sentence with respect to BERT vocabulary
tokenized_text = [tokenizer.tokenize(sent) for sent in sentences]

In [8]:
# Convert all sequences to BERT ids
# NOTE: BERT has a maximum sequence length of 512, so many review texts have
# been truncated
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_text],
                          maxlen = 512, dtype = "long", truncating = "post", padding = "post")

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (697 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (655 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (549 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (544 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (595 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length f

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (530 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (552 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (616 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (572 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (532 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length f

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (808 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (692 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (679 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (672 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (552 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length f

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (735 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (538 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (840 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (936 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (577 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length f

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (633 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (972 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (638 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (715 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (531 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length f

...

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (864 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (830 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (670 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (859 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (734 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length f

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1124 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (921 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (534 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (656 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (530 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length 

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (617 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (561 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (561 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (583 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (604 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length f

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1069 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (595 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (732 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (521 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (604 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length 

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (601 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (787 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (528 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (678 > 512). Running this sequence through BERT will result in indexing errors


In [9]:
# Attention masks for BERT, in our case, we wish to predict all words, so no 
# mask needed
attentions = []
for seq in input_ids:
  attentions.append([float(i > 0) for i in seq])

In [13]:
# Split training and validation data 8:2
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, train_data.stars, random_state = 2019,
                                                                                    test_size = 0.2)
train_masks, validation_masks, _, _ = train_test_split(attentions, input_ids, random_state = 2019, test_size = 0.2)

# Turn all data into tensors
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

# Set up iterators for tensors
batch_size = 64
train_tensor = TensorDataset(train_inputs, train_masks, train_labels)
validation_tensor = TensorDataset(validation_inputs, validation_masks, validation_labels)
train_sampler = RandomSampler(train_tensor)
validation_sampler = SequentialSampler(validation_tensor)
train_dataloader = DataLoader(train_tensor, sampler = train_sampler, batch_size = batch_size)
validation_dataloader = DataLoader(validation_tensor, sampler = validation_sampler,
                                   batch_size = batch_size)

In [14]:
model = BertForSequenceClassification.from_pretrained("bert-base-cased", num_labels = 5)
#model.cuda()

In [ ]:
parameters = list(model.named_parameters())
no_decay = ["bias", "gamma", "beta"]
grouped_paras = [
                 {"params": [p for n, p in parameters if not any(nd in n for nd in no_decay)],
                  "weight_decay_rate": 0.01},
                 {"params": [p for n, p in parameters if any(nd in n for nd in no_decay)],
                  "weight_decay_rate": 0.0}
]

def calc_acc(preds, labels):
  pred_flatten = np.argmax(preds, axis = 1).flatten()
  labels_flatten = labels.flatten()
  return np.sum(pred_flatten == labels_flatten) / len(labels.flatten)


optimizer = BertAdam(grouped_paras, lr = 3e-5, warmup = 0.01)

train_loss_list = []
epochs = 4

for _ in trange(epochs, desc = "Epoch"):
  model.train()
  train_loss = 0
  number_train_examples, number_train_steps = 0, 0
  for step, batch in enumerate(train_dataloader):
    #batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_masks, b_labels = batch
    optimizer.zero_grad()
    print("Step1")
    loss = model(b_input_ids, token_type_ids = None, attention_mask = b_masks,
                 labels = b_labels)
    print("Step2")
    train_loss_list.append(loss.item())
    loss.backward()
    print("Step3")
    optimizer.step()
    print("Step4")
    train_loss += loss.item()
    number_train_examples += b_input_ids.size(0)
    number_train_steps += 1

  model.eval()
  eval_loss, eval_acc = 0, 0
  number_eval_examples, number_eval_steps = 0, 0
  for batch in validation_dataloader:
    #batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_masks, b_labels = batch
    with torch.no_grad():
      logits = model(b_input_ids, token_type_ids = None, attention_mask = b_masks)
      logits = logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()
      tmp_eval_accuracy = calc_acc(logits, label_ids)
      eval_acc += tmp_eval_accuracy
      number_eval_steps += 1
    

t_total value of -1 results in schedule not being applied

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Step1
